# GQN With DGF - Pool Representation and 8 Generative Steps

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from gqn import GenerativeQueryNetwork, partition
from dataset import GQN_Dataset
from torch.distributions import Normal
import shutil
import os
import numpy as np
import cv2
import torch
import glob

In [ ]:
df = pd.read_csv("main.log")
print("Total steps: {}".format(df["step"].tolist()[-1]))
models = glob.glob("checkpoints/checkpoint_model_*.pth")
models.sort(key=lambda x: os.path.getmtime(x))
print("Last model checkpoint: {}".format(models[-1]))

In [ ]:
df = df[df["step"]>200000]
step = df["step"].to_list()
ll = df["ll"].to_list()
kl = df["kl"].to_list()
elbo = df['elbo'].to_list()
sigma = df['sigma'].to_list()
mu = df['mu'].to_list()
kl_av = []
ll_av = []
elbo_av = []

average_step = 100
for i in range(average_step, len(step)):
    kl_av.append(sum(kl[i-average_step:i])/average_step)
    ll_av.append(sum(ll[i-average_step:i])/average_step)
    elbo_av.append(sum(elbo[i-average_step:i])/average_step)
step = step[average_step:]
sigma = sigma[average_step:]
mu =mu[average_step:]

plt.figure(num=None, figsize=(18, 12))
plt.subplot(231)
plt.plot(step, elbo_av)
plt.subplot(232)
plt.plot(step, ll_av)
plt.subplot(233)
plt.plot(step, kl_av)

plt.subplot(234)
plt.plot(step, mu)
plt.subplot(235)
plt.plot(step, sigma)
plt.show()

In [ ]:
device = torch.device("cpu")

valid_dataset = GQN_Dataset(root_dir="data/rooms_ring_camera/", train=False)
x, v = valid_dataset[1]
x = x.view((1, *x.shape))
v = v.view((1, *v.shape))
x, v = x.to(device), v.to(device)
max_m=5
x, v, x_q, v_q = partition(x, v, max_m, 5)
batch, *_ = x.shape

model = GenerativeQueryNetwork(x_dim=3, v_dim=7, r_dim=256, h_dim=128, dgf_dim=256, z_dim=64, L=8, pool=True).to(device)
model.train()
checkpoint = torch.load(models[-1])
# checkpoint = torch.load("./checkpoints/checkpoint_model_50000.pth") #
model.load_state_dict(checkpoint)

x_mu, bn1_stats, bn2_stats = model.sample(x, v, v_q)
# x_mu, r, kl = model(x, v, x_q, v_q)
x_mu = x_mu.detach().cpu().numpy()
context_x = x.cpu().numpy()
query_x = x_q.cpu().numpy()

context_x = np.moveaxis(context_x, [2, 3, 4], [-1, 2, 3])
query_x = np.moveaxis(query_x, [1, 2, 3], [-1, -3, -2])
x_mu = np.moveaxis(x_mu, [1, 2, 3], [-1, -3, -2])

n = 36
for i in range(n):
    full = np.concatenate(context_x[i], axis=1)
    space = np.ones(shape=(full.shape[0], 10, 3))
    full = np.concatenate([full, space, x_mu[i], space, query_x[i]], axis=1)
    plt.figure(num=None, figsize=(18, 6))
    plt.imshow(full)
    print(i)
    plt.show()



In [ ]:
i = 9

In [ ]:
delta = 0.1
new_images = 7
change_n = 256
batch = 1

_, _, x_dims, h, w = x.shape

dgf = model.get_dgf(x, v, bn1_stats)[i]
dgf = dgf.cpu().detach().numpy()

view_q = v_q[i].cpu().detach().numpy()
view_q = np.reshape(view_q, (1,  view_q.shape[0]))
view_q = torch.tensor(view_q)

biases = model.input_layer.bias.cpu().detach().numpy()

deltas = [delta * (j - new_images//2) for j in range(new_images)]
for neuron_i in range(change_n):
    delta_set = []
    images = []
    for cur_delta in deltas:
        new_dgf = np.copy(dgf)
        new_dgf.astype(np.float16)
        new_dgf[neuron_i] += cur_delta
        delta_set.append("{:.5}".format(new_dgf[neuron_i]))
        new_dgf = np.expand_dims(new_dgf, axis=0)
        new_dgf = torch.tensor(new_dgf)

        device = torch.device("cpu")
        new_dgf, view_q = new_dgf.to(device), view_q.to(device)
        torch.manual_seed(1)
        x_mu = model.sample_from_dgf((batch, h, w), new_dgf, view_q, bn2_stats)
        x_mu = x_mu.detach().cpu().numpy()
        images.append(x_mu)

    images = np.array(images)
    images = np.reshape(images, (new_images, x_dims, h, w))
    images = np.moveaxis(images, [1, 2, 3], [-1, -3, -2])
    images = np.concatenate(images, axis=1)
    print("i: {}, bias: {:.4}".format(neuron_i+1, biases[neuron_i]))
    print("      ","        ".join(delta_set))
    plt.figure(num=None, figsize=(21,6))
    plt.imshow(images)
    plt.show()


In [ ]:
dgf_random = np.random.uniform(-0.05, 0.03, (1, 256))
# dgf_manual = np.random.normal(0, 0.05, (1, 256))

i=9

In [ ]:
i=25
dgf_random = model.get_dgf(x, v, bn1_stats)[i]
dgf_random = dgf_random.cpu().detach().numpy()
dgf_random = np.expand_dims(dgf_random, axis=0)

In [ ]:
_, _, x_dims, h, w = x.shape

# dgf_manual = np.zeros((1, 256), dtype=np.float32)
# dgf_manual = np.random.normal(0, 0.02, (1, 256))
# dgf_manual = np.random.uniform(-0.0, 0.05, (1, 256))

dgf_manual=dgf_random.astype(np.float32)
print(dgf_manual.dtype)



dgf_manual = torch.tensor(dgf_manual)
view_q = v_q[i].cpu().detach().numpy()
view_q = np.reshape(view_q, (1,  view_q.shape[0]))
view_q = torch.tensor(view_q)
print(dgf_manual.shape)
print(view_q.shape)

dgf_manual, view_q = dgf_manual.to(device), view_q.to(device)
torch.manual_seed(2)
x_mu = model.sample_from_dgf((1, h, w), dgf_manual, view_q, bn2_stats)
x_mu = x_mu.detach().cpu().numpy()[0]
x_mu = np.moveaxis(x_mu, [0, 1, 2], [2, 0, 1])
print(x_mu.shape)
plt.imshow(x_mu)


In [ ]:
i=9
dgf_manual = np.copy(dgf_random)
dgf_manual[0, 10] = 0.052
dgf_manual[0, 51] = 0.0454
dgf_manual[0, 120] = 0.0786
dgf_manual[0, 142] = 0.053
dgf_manual[0, 144] = 0.042
dgf_manual[0, 169] = 0.0534
dgf_manual[0, 207] = 0.0566
dgf_manual[0, 231] = 0.04615
dgf_manual[0, 234] = 0.085





dgf_manual=dgf_manual.astype(np.float32)
print(dgf_manual.dtype)



dgf_manual = torch.tensor(dgf_manual)
view_q = v_q[i].cpu().detach().numpy()
view_q = np.reshape(view_q, (1,  view_q.shape[0]))
view_q = torch.tensor(view_q)
print(dgf_manual.shape)
print(view_q.shape)

dgf_manual, view_q = dgf_manual.to(device), view_q.to(device)
torch.manual_seed(2)
x_mu = model.sample_from_dgf((1, h, w), dgf_manual, view_q, bn2_stats)
x_mu = x_mu.detach().cpu().numpy()[0]
x_mu = np.moveaxis(x_mu, [0, 1, 2], [2, 0, 1])
print(x_mu.shape)
plt.imshow(x_mu)

In [ ]:
#GRAY FLOOR
dgf_manual = np.copy(dgf_random)
dgf_manual[0, 8] = 0.16
dgf_manual[0, 249] = 0.13635


dgf_manual=dgf_manual.astype(np.float32)
print(dgf_manual.dtype)



dgf_manual = torch.tensor(dgf_manual)
view_q = v_q[i].cpu().detach().numpy()
view_q = np.reshape(view_q, (1,  view_q.shape[0]))
view_q = torch.tensor(view_q)
print(dgf_manual.shape)
print(view_q.shape)

dgf_manual, view_q = dgf_manual.to(device), view_q.to(device)
torch.manual_seed(2)
x_mu = model.sample_from_dgf((1, h, w), dgf_manual, view_q, bn2_stats)
x_mu = x_mu.detach().cpu().numpy()[0]
x_mu = np.moveaxis(x_mu, [0, 1, 2], [2, 0, 1])
print(x_mu.shape)
plt.imshow(x_mu)

In [ ]:
#YELLOW FLOOR
dgf_manual = np.copy(dgf_random)
dgf_manual[0, 15] = 0.1472
# dgf_manual[0, 54] = 0.1196
# dgf_manual[0, 65] = 0.32224
# dgf_manual[0, 136] = 0.18346



dgf_manual=dgf_manual.astype(np.float32)
print(dgf_manual.dtype)



dgf_manual = torch.tensor(dgf_manual)
view_q = v_q[i].cpu().detach().numpy()
view_q = np.reshape(view_q, (1,  view_q.shape[0]))
view_q = torch.tensor(view_q)
print(dgf_manual.shape)
print(view_q.shape)

dgf_manual, view_q = dgf_manual.to(device), view_q.to(device)
torch.manual_seed(2)
x_mu = model.sample_from_dgf((1, h, w), dgf_manual, view_q, bn2_stats)
x_mu = x_mu.detach().cpu().numpy()[0]
x_mu = np.moveaxis(x_mu, [0, 1, 2], [2, 0, 1])
print(x_mu.shape)
plt.imshow(x_mu)

In [ ]:
#ORANGE WALLS
dgf_manual = np.copy(dgf_random)
# dgf_manual[0, 44] = 0.21
# dgf_manual[0, 44] = 0.31
dgf_manual[0, 109] = 0.12958
# dgf_manual[0, 65] = 0.32224
# dgf_manual[0, 136] = 0.18346



dgf_manual=dgf_manual.astype(np.float32)
print(dgf_manual.dtype)



dgf_manual = torch.tensor(dgf_manual)
view_q = v_q[i].cpu().detach().numpy()
view_q = np.reshape(view_q, (1,  view_q.shape[0]))
view_q = torch.tensor(view_q)
print(dgf_manual.shape)
print(view_q.shape)

dgf_manual, view_q = dgf_manual.to(device), view_q.to(device)
torch.manual_seed(2)
x_mu = model.sample_from_dgf((1, h, w), dgf_manual, view_q, bn2_stats)
x_mu = x_mu.detach().cpu().numpy()[0]
x_mu = np.moveaxis(x_mu, [0, 1, 2], [2, 0, 1])
print(x_mu.shape)
plt.imshow(x_mu)

In [ ]:
#VIOLET WALLS
dgf_manual = np.copy(dgf_random)
dgf_manual[0, 51] = 0.1454
dgf_manual[0, 87] = 0.20423

dgf_manual=dgf_manual.astype(np.float32)
print(dgf_manual.dtype)



dgf_manual = torch.tensor(dgf_manual)
view_q = v_q[i].cpu().detach().numpy()
view_q = np.reshape(view_q, (1,  view_q.shape[0]))
view_q = torch.tensor(view_q)
print(dgf_manual.shape)
print(view_q.shape)

dgf_manual, view_q = dgf_manual.to(device), view_q.to(device)
torch.manual_seed(2)
x_mu = model.sample_from_dgf((1, h, w), dgf_manual, view_q, bn2_stats)
x_mu = x_mu.detach().cpu().numpy()[0]
x_mu = np.moveaxis(x_mu, [0, 1, 2], [2, 0, 1])
print(x_mu.shape)
plt.imshow(x_mu)

In [ ]:
#ORANGE WALLS
dgf_manual = np.copy(dgf_random)
dgf_manual[0, 205] = 0.15035
# dgf_manual[0, 224] = 0.094652
# dgf_manual[0, 44] = 0.31
# dgf_manual[0, 109] = 0.12958
# dgf_manual[0, 65] = 0.32224
# dgf_manual[0, 136] = 0.18346



dgf_manual=dgf_manual.astype(np.float32)
print(dgf_manual.dtype)



dgf_manual = torch.tensor(dgf_manual)
view_q = v_q[i].cpu().detach().numpy()
view_q = np.reshape(view_q, (1,  view_q.shape[0]))
view_q = torch.tensor(view_q)
print(dgf_manual.shape)
print(view_q.shape)

dgf_manual, view_q = dgf_manual.to(device), view_q.to(device)
torch.manual_seed(2)
x_mu = model.sample_from_dgf((1, h, w), dgf_manual, view_q, bn2_stats)
x_mu = x_mu.detach().cpu().numpy()[0]
x_mu = np.moveaxis(x_mu, [0, 1, 2], [2, 0, 1])
print(x_mu.shape)
plt.imshow(x_mu)

In [ ]:
i=14
delta = 0.1
new_images = 7
change_n = 256
batch = 1

_, _, x_dims, h, w = x.shape

dgf = model.get_dgf(x, v, bn1_stats)[i]
dgf = dgf.cpu().detach().numpy()

view_q = v_q[i].cpu().detach().numpy()
view_q = np.reshape(view_q, (1,  view_q.shape[0]))
view_q = torch.tensor(view_q)

biases = model.input_layer.bias.cpu().detach().numpy()

deltas = [delta * (j - new_images//2) for j in range(new_images)]
for neuron_i in range(change_n):
    delta_set = []
    images = []
    for cur_delta in deltas:
        new_dgf = np.copy(dgf)
        new_dgf.astype(np.float16)
        new_dgf[neuron_i] += cur_delta
        delta_set.append("{:.5}".format(new_dgf[neuron_i]))
        new_dgf = np.expand_dims(new_dgf, axis=0)
        new_dgf = torch.tensor(new_dgf)

        device = torch.device("cpu")
        new_dgf, view_q = new_dgf.to(device), view_q.to(device)
        torch.manual_seed(1)
        x_mu = model.sample_from_dgf((batch, h, w), new_dgf, view_q, bn2_stats)
        x_mu = x_mu.detach().cpu().numpy()
        images.append(x_mu)

    images = np.array(images)
    images = np.reshape(images, (new_images, x_dims, h, w))
    images = np.moveaxis(images, [1, 2, 3], [-1, -3, -2])
    images = np.concatenate(images, axis=1)
    print("i: {}, bias: {:.4}".format(neuron_i+1, biases[neuron_i]))
    print("      ","        ".join(delta_set))
    plt.figure(num=None, figsize=(21,6))
    plt.imshow(images)
    plt.show()


In [ ]:
###
### MORE EFFECTIVE CODE, BUT WITHOUT SEED!!!!!!!!!
###

delta = 0.1
new_images = 7
change_n = 256
batch = new_images*change_n

_, _, x_dims, h, w = x.shape

dgf = model.get_dgf(x, v, bn1_stats)[i]
dgf = dgf.cpu().detach().numpy()
dgf_dims = dgf.shape[0]
dgf = np.reshape(dgf, (1, dgf.shape[0]))
new_dgf = np.repeat(dgf, batch, axis=0)

deltas = np.zeros((batch, dgf_dims), dtype=np.float16)
delta_row = [delta * (j - new_images//2) for j in range(new_images)]
for j in range(0, change_n):
    deltas[j*new_images:j*new_images+new_images, j] = delta_row

new_dgf = new_dgf + deltas

view_q = v_q[i].cpu().detach().numpy()
view_q = np.reshape(view_q, (1,  view_q.shape[0]))
view_q = np.repeat(view_q, batch, axis=0)

view_q = torch.tensor(view_q)
new_dgf = torch.tensor(new_dgf)
print(new_dgf.shape)
print(view_q.shape)


device = torch.device("cpu")
new_dgf, view_q = new_dgf.to(device), view_q.to(device)
x_mu = model.sample_from_dgf((batch, h, w), new_dgf, view_q, bn2_stats)
x_mu = x_mu.detach().cpu().numpy()
x_mu = np.moveaxis(x_mu, [1, 2, 3], [-1, -3, -2])
new_dgf_np = new_dgf.detach().cpu().numpy()

for j in range(change_n):
    change_set = x_mu[j*new_images:j*new_images+new_images]
    delta_set = new_dgf_np[j*new_images:j*new_images+new_images, j]
    images = np.concatenate(change_set, axis=1)
    delta_set = delta_set.astype(np.float16)
    print("      ", "         ".join(map(str, delta_set)))
    plt.figure(num=None, figsize=(21,6))
    plt.imshow(images)
    plt.show()
